In [40]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [41]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    branch="master",
    repo_path="./langchain",
    file_filter=file_filter
)

documents = loader.load()
print(len(documents))

439


In [42]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings

emb = AzureOpenAIEmbeddings(
    model = "text-embedding-ada-002"
)
# AOAI の RateLimit で 429 を避けるため、ドキュメントの数を絞る
db = Chroma.from_documents(documents[:10], emb)

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI

prompt = ChatPromptTemplate.from_template(
    '''
    以下の文脈だけを参考にして、質問に答えてください。
    文脈: {context}
    質問: {question}
    '''
)

In [44]:
import os
from azure.identity import ClientSecretCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    ClientSecretCredential(
        client_id=os.getenv("client_id"),
        client_secret=os.getenv("client_secret"),
        tenant_id=os.getenv("tenant_id")
    ),
    "https://cognitiveservices.azure.com/.default"
)
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-02-15-preview",
    azure_ad_token_provider=token_provider,
    temperature=0
)

In [45]:
retriever = db.as_retriever()

In [46]:
# 最初の {"context": retriever, "question": RunnablePassthrough()} は RunnableParallel が効いている
# そのため、context と question は同時に処理される
# chain.invoke の引数は並行して retriever と RunnablePassthrough に渡される
chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

output = chain.invoke("LangChainについて教えてください。")
print(output)

LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。以下にLangChainの主な特徴を説明します。

---

### **LangChainの主な機能**
1. **開発（Development）**:
   - LangChainは、オープンソースのコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。
   - [LangGraph](https://langchain-ai.github.io/langgraph/)を使用して、ストリーミングや人間の介入をサポートする状態保持型エージェントを構築できます。

2. **プロダクション化（Productionization）**:
   - [LangSmith](https://docs.smith.langchain.com/)を使用して、アプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイメント（Deployment）**:
   - LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換するための[LangGraph Platform](https://docs.langchain.com/langgraph-platform)を提供しています。

---

### **LangChainのアーキテクチャ**
LangChainは複数のオープンソースライブラリで構成されています。
- **`langchain-core`**: チャットモデルやその他のコンポーネントの基本的な抽象化を提供。
- **統合パッケージ**: 重要な統合（例: `langchain-openai`, `langchain-anthropic`など）は軽量パッケージとして分割され、LangChainチームと統合開発者によって共同管理されています。
- **`langchain`**: チェーン、エージェント、検索戦略など、アプリケーションの認知アーキテクチャを構成する要素。
- **`langchain-community`**: コミュニティによって維持されるサ

In [47]:
chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | RunnablePassthrough.assign( ans = prompt | llm | StrOutputParser())

output = chain.invoke("LangChainについて教えてください。")
print(output['ans'])

LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。以下にLangChainの主な特徴と機能を説明します。

---

### **LangChainの特徴**
1. **開発（Development）**:
   - LangChainは、オープンソースのコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。
   - [LangGraph](https://langchain-ai.github.io/langgraph/)を使用して、ストリーミングや人間の介入をサポートする状態管理エージェントを構築可能です。

2. **プロダクション化（Productionization）**:
   - [LangSmith](https://docs.smith.langchain.com/)を使用して、アプリケーションの検査、監視、評価を行い、継続的な最適化と信頼性のあるデプロイを実現します。

3. **デプロイメント（Deployment）**:
   - LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換するための[LangGraph Platform](https://docs.langchain.com/langgraph-platform)を提供します。

---

### **LangChainのアーキテクチャ**
LangChainは複数のオープンソースライブラリで構成されています：
- **`langchain-core`**: チャットモデルやその他のコンポーネントの基本的な抽象化を提供。
- **統合パッケージ**: 主要な統合（例: `langchain-openai`, `langchain-anthropic`）は軽量パッケージとして提供され、LangChainチームと統合開発者によって共同管理されています。
- **`langchain`**: チェーン、エージェント、情報検索戦略など、アプリケーションの認知アーキテクチャを構成。
- **`langchain-community`**: コミュニティによって管理されるサードパ

## HyDE というテクニック
確かに、RAGの仕組み上疑問ではあったが、なぜ質問に対して類似する文章をベクトルDBから探すんだろうと思っていた。
探しているのは回答なので、一度LLMに回答を出力させてからそれに類似するチャンクをDBから引っ張ってくるほうが分布としては近いんじゃないか？というのがHyDE。
つまり、ベクトルDBに対する検索クエリの工夫によって精度向上を目指すという話ね。


In [48]:
hypothetical_prompt = ChatPromptTemplate.from_template(
    '''
    次の質問に回答する一文を書いてください。
    質問: {question}
    '''
)
hypo_chain = hypothetical_prompt | llm | StrOutputParser()

In [49]:
# 本来ユーザからの質問が直接 retriever に渡るのを、hypo_chain の出力を渡すようにする
hyde_rag_chain = {
    "context": hypo_chain | retriever,
    "question": RunnablePassthrough()
} | RunnablePassthrough.assign( ans = hypothetical_prompt | llm | StrOutputParser())
output = hyde_rag_chain.invoke("LangChainについて教えてください。")
print(output['ans'])

LangChainは、LLM（大規模言語モデル）を活用したアプリケーションの開発を支援するフレームワークで、プロンプト管理、チェーン構築、外部データとの連携などを容易にするツールです。


## ユーザのクエリの違うパターンを LLM に生成させる
これにより、類似の質問が複数渡されるため、BootStrapのようなイメージで少し異なるコンテキストに対する retrieve 結果をもとに回答できる。

## 複数クエリを与える場合のチャンクの重みづけ
複数クエリのそれぞれで共通して参照されるチャンクは重みをつけることで、なるべくそのチャンクを重視した最終回答を生成させる。やり方としては、RRF値による並べ替え、並べ替え＋上位ｎ位以下の切り捨て、など

## 単一クエリの場合の参照チャンクのリランク
ユーザのクエリと近いベクトルを持つチャンクを複数引っ張ってきたのち、そのチャンクをリランクタスクにおいて精度の高い別のモデルによって再度並べ替えを行う

In [50]:
print(output['context'])

[Document(id='218069c7-6e2b-4733-821a-ba3579b9dcf1', metadata={'file_name': 'introduction.mdx', 'source': 'docs\\docs\\introduction.mdx', 'file_path': 'docs\\docs\\introduction.mdx', 'file_type': '.mdx'}, page_content='---\nsidebar_position: 0\nsidebar_class_name: hidden\n---\n\n# Introduction\n\n**LangChain** is a framework for developing applications powered by large language models (LLMs).\n\nLangChain simplifies every stage of the LLM application lifecycle:\n- **Development**: Build your applications using LangChain\'s open-source [components](/docs/concepts) and [third-party integrations](/docs/integrations/providers/).\nUse [LangGraph](/docs/concepts/architecture/#langgraph) to build stateful agents with first-class streaming and human-in-the-loop support.\n- **Productionization**: Use [LangSmith](https://docs.smith.langchain.com/) to inspect, monitor and evaluate your applications, so that you can continuously optimize and deploy with confidence.\n- **Deployment**: Turn your Lan

retrieve 用とリランク用に分けたり、retriever を複数にして LLM にルーティングさせたり、結局マルチエージェントとかTools、MCPっぽい構想なんだな(動的に手札を切らせる)

## 複数の Retriever
LLM による埋め込みの ベクトル DB 内で距離を測るよりも、TF-IDFなどの出現率ベースのベクトルのほうが専門分野に強かったりする。そのため、Embedding BaseなRetrieverと、TF-IDFベースを組み合わせたりする。

In [51]:
# TF-IDFも合わせてみるか
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents[:10]).with_config({"run_name": "bm25_retriever"})


In [52]:
hybrid_retriever = {
    "bm25_documents":bm25_retriever,
    "chroma_documents":retriever
} | xxxxx | rerank_process
# みたいな感じで、両方の retriever から出てきた Document を RRF に従って並べ替える
# こんな感じで、最終的には 1つの retriever にまとめることができる

NameError: name 'xxxxx' is not defined